In [1]:
!pip install -q datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers.git to /var/tmp/pip-req-build-sl5cq05c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /var/tmp/pip-req-build-sl5cq05c
  Resolved https://github.com/huggingface/transformers.git to commit 00c1d87a7d5c8dfb4554370983b5a3f7c069edd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
os.getcwd() 

'/home/subhendu-pg'

In [3]:
!ls

anaconda3  Anaconda3-2020.07-Linux-x86_64.sh  idefics_venv  idefic_venv


In [3]:
!pip install Pillow
!pip install matplotlib
from PIL import Image
import matplotlib.pyplot as plt 
image = Image.open('../../ChartQADataset/train/png/two_col_81284.png')
print(image)
plt.imshow(image)

FileNotFoundError: [Errno 2] No such file or directory: '../../ChartQADataset/train/png/two_col_81284.png'

In [5]:

import pandas as pd
train_json=pd.read_json("../../ChartQADataset/val/val_augmented.json")


In [6]:
train_json

,imgname,query,label
0,multi_col_20933.png,Who portrayed Jon Snow?,Kit Harington
1,multi_col_20933.png,Who played Mother of Dragons Daenerys Targaryen?,Emilia Clarke
2,multi_col_20933.png,What was the estimated net worth of Peter Dink...,16
3,multi_col_972.png,What is the most popular cross-platform mobile...,React Native
4,multi_col_60377.png,When did the population of the Dominican Repub...,2008
...,...,...,...
955,two_col_4757.png,What was the most valuable British-Dutch oil a...,Shell
956,two_col_4757.png,What was the only brand in the top ten with he...,Shell
957,two_col_4757.png,What was the second most valuable oil and gas ...,BP
958,two_col_23389.png,Where did migrants arrive from January to Dece...,Pozzallo


In [4]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# checkpoint = "HuggingFaceM4/tiny-random-idefics"
checkpoint = "HuggingFaceM4/idefics-9b"

# Here we skip some special modules that can't be quantized properly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

processor = AutoProcessor.from_pretrained(checkpoint, use_auth_token=False)
# Simply take-off the quantization_config arg if you want to load the original model
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

In [ ]:
print(model)

In [ ]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

In [ ]:
url = "https://factly.in/wp-content/uploads//2020/09/India%E2%80%99s-GDP-growth-rate_GDP-growth-comparison-Business-Today-1024x698.jpg"
prompts = [
    # "Instruction: provide an answer to the question. Use the image to answer.\n",
    image,
    "Question: What is this image about ? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=5)

In [ ]:
# check generation before finetuning

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Mona_Lisa_%28copy%2C_Hermitage%29.jpg/224px-Mona_Lisa_%28copy%2C_Hermitage%29.jpg"
prompts = [
    url,
    "Question: Who is the lady in the painting ? Answer:",
]
check_inference(model, processor, prompts, max_new_tokens=100)
# It looks like the model is already aware of pokemon - but it could be more specific, and less repetitive

In [ ]:
def convert_to_rgb(image):
    # `image.convert("RGB")` would only work for .jpg images, as it creates a wrong background
    # for transparent images. The call to `alpha_composite` handles this case
    if image.mode == "RGB":
        return image

    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite


In [ ]:
def ds_transforms(example_batch,path):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std

    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std),
    ])

    prompts = []
    for i in range(len(example_batch['query'])):
        # We split the captions to avoid having very long examples, which would require more GPU ram during training
        caption = example_batch['query'][i].split(".")[0]
        img_name=example_batch['imgname'][i]
        # image_url="train/png/"+img_name
        image_url=path+img_name
        image = Image.open(image_url)
        prompts.append(
            [
                #
                image,
                f"Question: What's on the picture? Answer: This is {example_batch['label'][i]}. {caption}",
            ],
        )

    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)

    inputs["labels"] = inputs["input_ids"]

    return inputs

In [26]:
!pip install datasets

# from datasets import dataset_utils

NameError: name 'datasets' is not defined

In [21]:
!pip install datasets
from datasets import Dataset
class ChartQADataset(Dataset):
    """ChartQA Dataset."""

    def __init__(self, json_file, root_dir='./', transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.

        """
        self.tacos_df = pd.read_json(json_file)
        self.root_dir = root_dir
        self.transform = transform
        self.image_name=self.tacos_df['imgname']
        self.sentences = self.tacos_df['query']
        #self.labels = self.tacos_df['Sentiment']
        self.text_labels = self.tacos_df['label']
        #self.abstracts = self.tacos_df['Aspect']

    def __len__(self):
        return len(self.tacos_df)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        image=self.image_name[idx]
        sentence = self.sentences[idx]
        #label = self.labels[idx]
        text_label = self.text_labels[idx]
        #print('text label',text_label)
        #abstract = self.abstracts[idx]

        sample = {'image':image, 'sentence': sentence,  'text_label': text_label}

        return sample


   ---------------------------------------- 0.0/233.4 kB ? eta -:--:--
   ----- --------------------------------- 30.7/233.4 kB 660.6 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/233.4 kB 880.9 kB/s eta 0:00:01
   ----------------------------- ---------- 174.1/233.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 233.4/233.4 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.6 MB 2.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.6 MB 2.6 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.6 MB 2.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.6 MB 2.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.6 MB 2.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.6 MB 2.6 MB/s eta 0:00:01
   ---------------- ----------------------- 0.7/1.6 MB 2.5 MB/s eta 0:00:01
   -----

ModuleNotFoundError: No module named 'dataset'

In [ ]:
tacos_dataset_train = ChartQADataset(csv_file='../../ChartQADataset/train/train_augmented.json')
tacos_dataset_val = ChartQADataset(csv_file='../../ChartQADataset/val/val_augmented.json')


In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_dict(
        {"image":list(tacos_dataset_train.image_name),"sentence": list(tacos_dataset_train.sentences), "text_label": list(tacos_dataset_train.text_labels)})
dataset_val = Dataset.from_dict(
        {"image":list(tacos_dataset_val.image_name),"sentence": list(tacos_dataset_val.sentences), "text_label": list(tacos_dataset_val.text_labels)})

In [11]:
!pip install sklearn
from sklearn.model_selection import train_test_split
train_data = train_json
train_ds, eval_ds = train_test_split(train_data, test_size=0.002)
train_ds
eval_ds
train_ds=

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)
model = get_peft_model(model, config)


In [ ]:
model.print_trainable_parameters()


In [ ]:
training_args = TrainingArguments(
    output_dir=f"{model_name}-output",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=20,
    logging_steps=20,
    max_steps=40,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    report_to=None,
    optim="paged_adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
)

trainer.train()